# Workshop: 📖🦙 NotebookLM Clone based on Llama Open Weights Models

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/notebook-llama/blob/main/workshop.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this workshop, we'll be building a podcast generator from a PDF file.

- 🌠 Workshop slides: https://go.union.ai/workshop-notebook-lm-clone
- 📱 Example app: https://shy-sun-51a14.apps.serverless-1.us-east-2.s.union.ai/

## 🧱 Setup

Before running this notebook, make sure you follow the prerequisites in the
[README](https://github.com/unionai-oss/notebook-llama/blob/main/README.md).

In [34]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/notebook-llama
    %cd notebook-llama
    !pip install -r requirements.txt

Authenticate this notebook session with Union:

In [ ]:
!union create login --auth device-flow --serverless

Next, create a Huggingface API key secret: https://huggingface.co/settings/tokens.

Run the cell below to create the secret on Union. You'll be prompted to paste
the string into the input field.

In [ ]:
!union create secret huggingface_api_key

If you don't already, request access to the [Llama 3.2 3B model](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
and [Llama 3.2 1B model](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct).

To make sure the secret is created, run the cell below:





In [ ]:
!union get secret

Finally, register the `notebook_llama` tasks and workflows:

In [ ]:
!union register notebook_llama

## 🎛️ Setup a `UnionRemote` object

First, let's create a `UnionRemote` object, which will allow us to interact with
the Union platform.

In [20]:
import union
import rich

remote = union.UnionRemote(
    default_project="default",
    default_domain="development",
    interactive_mode_enabled=False,
)

def print_prompt(title: str, prompt: str):
    return rich.panel.Panel(prompt, title=title, width=120, border_style="red")

def print_text(title: str, text: str):
    return rich.panel.Panel(text, title=title, width=120, border_style="yellow")

## 📄 Extract Text from PDF

The first step in our application is to extract text from the PDF file. We
expect the output of this step to be in a raw text format, with all of the
formatting artifacts of the PDF file removed.

In this step we'll use a small language model to perform the extraction and
clearning.

In [ ]:
from notebook_llama.preprocess_pdf import pdf_to_text, SYS_PROMPT as EXTRACT_PDF_SYS_PROMPT

rich.print(print_prompt("PDF Extraction System Prompt", EXTRACT_PDF_SYS_PROMPT))

pdf_to_text_execution = remote.execute(
    pdf_to_text,
    inputs={"pdf_path": "https://arxiv.org/pdf/2503.14233"},
)
pdf_to_text_execution

We can take a look at the extracted text in the cell below:

In [ ]:
pdf_to_text_execution = remote.wait(pdf_to_text_execution)
extracted_text = pdf_to_text_execution.outputs["o0"]

with open(extracted_text) as f:
    text = f.read()

print_text("Extracted Text", text)

## 📝 Write Transcript

Next, we'll use a larger language model to write a draft of the transcript. This
should output a file containing the contents of the transcript draft.

In [ ]:
from notebook_llama.write_transcript import write_transcript, SYSTEM_PROMPT as WRITE_TRANSCRIPT_SYSTEM_PROMPT

rich.print(print_prompt("Write Transcript System Prompt", WRITE_TRANSCRIPT_SYSTEM_PROMPT))

write_transcript_execution = remote.execute(
    write_transcript,
    inputs={"pdf_text": extracted_text},
    version="latest",
)

write_transcript_execution

Once the execution is complete, we can take a look at the contents of the
transcript draft below:

In [ ]:
write_transcript_execution = remote.wait(write_transcript_execution)
raw_transcript_file = write_transcript_execution.outputs["o0"]

with open(raw_transcript_file) as f:
    raw_transcript = f.read()

print_text("Raw Transcript", raw_transcript)

## ✍️ Rewrite Transcript

Next, we'll use the same language model to rewrite the transcript to punch up
the script a little bit.

In [ ]:
from notebook_llama.rewrite_transcript import rewrite_transcript, SYSTEM_PROMPT as REWRITE_TRANSCRIPT_SYSTEM_PROMPT

rich.print(print_prompt("Rewrite Transcript System Prompt", REWRITE_TRANSCRIPT_SYSTEM_PROMPT))

rewrite_transcript_execution = remote.execute(
    rewrite_transcript,
    inputs={"transcript": raw_transcript_file},
    version="latest",
)

rewrite_transcript_execution

Let's take a look at the rewritten transcript below:

In [ ]:
import json
import rich.markdown

rewrite_transcript_execution = remote.wait(rewrite_transcript_execution)
rewritten_transcript_file = rewrite_transcript_execution.outputs["o0"]

with open(rewritten_transcript_file) as f:
    rewritten_transcript = json.load(f)

text = ""
for speaker, line in rewritten_transcript:
    text += f"**{speaker}**: {line}\n\n"

print_text("Rewritten Transcript", rich.markdown.Markdown(text))

## Generate Podcast

Finally, we'll use a text-to-speech model to generate the podcast audio of
our two speakers using the rewritten transcript.

In [ ]:
from notebook_llama.generate_podcast import generate_podcast, speaker1_description, speaker2_description

rich.print(print_prompt("Speaker 1 Description", speaker1_description))
rich.print(print_prompt("Speaker 2 Description", speaker2_description))

generate_podcast_execution = remote.execute(
    generate_podcast,
    inputs={"clean_transcript": rewritten_transcript_file},
    version="latest",
)

generate_podcast_execution

In [ ]:
from IPython.display import Audio

generate_podcast_execution = remote.wait(generate_podcast_execution)
podcast_file = generate_podcast_execution.outputs["o0"]

podcast_file.download()
Audio(podcast_file.path)


## 🚀 Serving an application

In the final part of this workshop, we'll serve the application to the public.
This will run the entire workflow we just ran in a single click through a nice
UI.

To do this, we need to create a Union API key called `notebook-llama-workshop` for app serving.

In [ ]:
# Make sure to store this key somewhere secure.
!union create api-key admin --name notebook-llama-workshop

You can list the api keys you have with:

In [ ]:
!union get api-key admin

Then, create an Union API key secret that we'll use for this workshop:

In [ ]:
!union create secret union_api_key

In [ ]:
from app import app

remote.deploy_app(app);

## ⭐️ Conclusion

Congrats! You've just deployed your first Compound AI System. We used multiple
models of different sizes and modalities to create a pipeline that generates
podcast audio from a PDF file.